<a href="https://colab.research.google.com/github/bpratham2001/Servilia/blob/main/notebooks/Wine_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
#!pip install kagglehub[pandas-datasets]
!pip install opendatasets

In [13]:
import opendatasets as od
import pandas as pd
import numpy as np
flag = False
#checked
#mysarahmadbhat/wine-tasting // winemag-data-130k-v2.csv \\ zynicide/wine-reviews is better

#potential sources
# https://en.wikipedia.org/wiki/Wine_tasting_descriptors
# https://www.wineenthusiast.com/toplists/wine/
# https://en.wikipedia.org/wiki/Wine_tasting

od.download("https://www.kaggle.com/datasets/zynicide/wine-reviews", force=flag) # winemag-data_first150k.csv, winemag-data-130k-v2.csv
file =('wine-reviews/winemag-data_first150k.csv')
df1 = pd.read_csv(file)
file =('wine-reviews/winemag-data-130k-v2.csv')
df2 = pd.read_csv(file)

od.download("https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset", force=flag) # wines_SPA.csv
file =('spanish-wine-quality-dataset/wines_SPA.csv')
df3 = pd.read_csv(file)

od.download("https://www.kaggle.com/datasets/elvinrustam/wine-dataset", force=flag) # WineDataset.csv
file =('wine-dataset/WineDataset.csv')
df4 = pd.read_csv(file)

file =('wine-tastings-guide.txt') # https://www.wine-tastings-guide.com/wine-descriptions.html
df5 = pd.read_csv(file, sep = ' - ')
file =('grapes.txt')
df6 = pd.read_csv(file, sep = ' - ')
file =('FR-IT.txt')
df7 = pd.read_csv(file, sep = ' - ')

Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews


100%|██████████| 50.9M/50.9M [00:01<00:00, 29.4MB/s]



Dataset URL: https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset


100%|██████████| 46.4k/46.4k [00:00<00:00, 2.24MB/s]


Dataset URL: https://www.kaggle.com/datasets/elvinrustam/wine-dataset


100%|██████████| 308k/308k [00:00<00:00, 905kB/s]


<ipython-input-13-9d26ab70dc83>:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df5 = pd.read_csv(file, sep = ' - ')
<ipython-input-13-9d26ab70dc83>:30: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df6 = pd.read_csv(file, sep = ' - ')
<ipython-input-13-9d26ab70dc83>:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df7 = pd.read_csv(file, sep = ' - ')


In [14]:
print(df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]+df6.shape[0]+df7.shape[0])

289778


In [15]:
df1.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
df2.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
#df['DataFrame Column'] = df['DataFrame Column'].fillna(0)

# fix designation and regions in df1 and df2 // not fixable?
#print(df1[df1['designation'].isnull()])
#print(df2[df2['designation'].isnull()])
region = np.concatenate((df1[df1['region_1'].isnull()].winery.unique(), df2[df2['region_1'].isnull()].winery.unique()))
region = np.concatenate((region, df1[df1['region_2'].isnull()].winery.unique()))
region = set(np.concatenate((region, df2[df2['region_2'].isnull()].winery.unique())))
print(str(len(region)) + " wineries of unknown region \n")
#df1.loc[(df1.region_2.isnull()) & (df1.region_1.isnull())])

# fix countries in df1 and df2, fix variety in df2
countries = {'Tsililis':'Greece', 'Büyülübağ':'Turkey', 'Chilcas':'Chile',
             'Gotsa Family Wines':'Georgia', 'Barton & Guestier':'France',
             'Kakhetia Traditional Winemaking':'Georgia', 'Ross-idi':'Bulgaria',
             'Orbelus':'Bulgaria', 'St. Donat':'Hungary', 'Familia Deicas':'Uruguay','Bartho Eksteen':'South Africa',
             'Stone Castle':'Kosovo', 'Teliani Valley':'Georgia', 'Undurraga':'Chile', 'Mt. Beautiful':'New Zealand',
             'Neumeister':'Austria', 'Santa Ema':'Chile', 'Psagot':'Israel',
             'Newton Johnson':'South Africa', 'Ktima Voyatzi':'Greece', 'Lismore':'South Africa', 'Logodaj':'Bulgaria',
             'Somlói Vándor':'Hungary', 'Les Frères Dutruy':'Switzerland', 'El Capricho':'Uruguay',
             'Amiran Vepkhvadze':'Georgia'}

df2.at[38898, 'country'] = 'Canada'
df2.at[38898, 'region_1'] = 'Twenty Mile Bench'
df2.at[38898, 'region_2'] = 'Niagara Peninsula'
df2.at[86909, 'variety'] = 'Petite Syrah'
df2.at[86909, 'region_1'] = 'Maipo Valley'

for index, row in df1.iterrows():
    if df1.at[index, 'winery'] in countries:
      df1.at[index, 'country'] = countries[df1.at[index, 'winery']]
for index, row in df2.iterrows():
    if df2.at[index, 'winery'] in countries:
      df2.at[index, 'country'] = countries[df2.at[index, 'winery']]

# merge regions in df1 and df2, drop useless columns
for i, row in df1.iterrows():
  if df1.at[i, 'region_1'] != df1.at[i, 'region_2']:
    if str(df1.at[i, 'region_1']) == 'nan':
      df1.at[i, 'region_1'] = str(df1.at[i, 'region_2'])
    elif str(df1.at[i, 'region_2']) != 'nan':
      df1.at[i, 'region_1'] = (str(df1.at[i, 'region_1']) +", "+ str(df1.at[i, 'region_2']))
df1 = df1.drop('region_2', axis=1)
df1 = df1.drop('price', axis=1)
df1 = df1.drop('points', axis=1)

for i, row in df2.iterrows():
  if df2.at[i, 'region_1'] != df2.at[i, 'region_2']:
    if str(df2.at[i, 'region_1']) == 'nan':
      df2.at[i, 'region_1'] = str(df2.at[i, 'region_2'])
    elif str(df2.at[i, 'region_2']) != 'nan':
      df2.at[i, 'region_1'] = (str(df2.at[i, 'region_1']) +", "+ str(df2.at[i, 'region_2']))
df2 = df2.drop('region_2', axis=1)
df2 = df2.drop('taster_twitter_handle', axis=1)
df2 = df2.drop('price', axis=1)

# check df1/df2 stats
for i in df1.keys():
  print(i +" "+ str(df1[i].isnull().values.any()))
df1.isnull().values.any()
print("\n")
for i in df2.keys():
  print(i +" "+ str(df2[i].isnull().values.any()))

14017 wineries of unknown region 

id False
country False
description False
designation True
province True
region_1 False
variety False
winery False


id False
country False
description False
designation True
points False
province True
region_1 False
taster_name True
title False
variety False
winery False


In [16]:
for i, row in df3.iterrows():
  df3.at[i, 'country'] = "Spain"
df3 = df3.drop('price', axis=1)
for i in df3.keys():
  print(i +" "+ str(df3[i].isnull().values.any()))

winery False
wine False
year True
rating False
num_reviews False
country False
region False
type True
body True
acidity True


In [17]:
for i, row in df4.iterrows():
  if df4.at[i, 'Country'] == 'USA':
    df4.at[i, 'Country'] = 'US'
df4 = df4.drop('Price', axis=1)
df4 = df4.drop('Capacity', axis=1)
df4 = df4.drop('Unit', axis=1)
df4 = df4.drop('Per bottle / case / each', axis=1)
df4 = df4.drop('ABV', axis=1)
for i in df4.keys():
  print(i +" "+ str(df4[i].isnull().values.any()))

Title False
Description True
Grape True
Secondary Grape Varieties True
Closure True
Country True
Characteristics True
Type True
Region True
Style True
Vintage True
Appellation True


In [10]:
#target format
# how to train on df? yahma/alpaca-cleaned
df = pd.read_json("hf://datasets/yahma/alpaca-cleaned/alpaca_data_cleaned.json")
df.head()

,instruction,input,output
0,Give three tips for staying healthy.,,1. Eat a balanced and nutritious diet: Make su...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,An atom is the basic building block of all mat...
3,How can we reduce air pollution?,,There are several ways to reduce air pollution...
4,Pretend you are a project manager of a constru...,,I had to make a difficult decision when I was ...


In [24]:
from random import randint as r
# prompts
imp = ["Tell me more about", "Describe", "I would like to know more about", "I want to know more about",
       "Tell me what", "Explain", "Elaborate", "Enlighten me on", "Educate me on"]
what_singular = ["What is", "What's"]
what_plural = ["What are"]
how_singular = ["How does", "How is", "How can", "How come"]
how_plural = ["How do", "How are", "How can", "How come"]
where_singular = ["Where is", "What is the location of", "Where can I find", "How do I come across"]
where_plural = ["Where are", "What are the locations of"]
why_singular = ["Why does", "Why is", "Why would"]
why_plural = ["Why do", "Why are"]
pref = ["I like", "I prefer", "I enjoy", "I love", "I don't mind"]
rank = ["over", "instead of", "less than", "more than", "not as much as", "as much as",
        "almost as much as", "almost as little as", "as little as"]
npref = ["I hate", "I detest", "I can't stand", "I'm wary of", "I don't like"]
point = ["this", "that", "it"]
connect = ["and", "with", "also", "but", "to", "cum"]

df0 = pd.DataFrame(columns=['instruction', 'input', 'output'])

In [26]:
# functions to construct question/answers from Kaggle databases // incomplete
def df1_construct_in(index, topic):
  instruction = str(imp[r(0, 8)] + " " + topic + ".")
  pass

def df1_construct_out(row, topic):
  output = row['description'] + 'From ' + row['winery']
  if row['region_1'] != 'nan':
    output += ", in " + row['region_1'] + "," + row['country']
  else:
    output += ", in " + row['country']
  if str(row['designation']) != 'nan' and str(row['variety']) != 'nan':
    output += ", this " + row['variety']

In [32]:
# functions to construct question/answers from blogs/articles
def construct_full(row, rng):
  if rng == 0:
    instruction = str(imp[r(0, 8)] + " " +row['word']+".")
  elif rng == 1:
    instruction = str(what_singular[r(0, 1)] + " " +row['word']+"?")
  else:
    instruction = str(pref[r(0, 4)] + " " +row['word']+".")
  output = row['description']
  return instruction, output

In [33]:
df5.shape[0]+df6.shape[0]+df7.shape[0]

87

In [ ]:
for i,r in df1.iterrows():
  print(r['country'])
  break